# SQL Assignment

In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

## Sample Code

In [4]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
Wall time: 209 ms


,Movie_Name
0,Mastizaade
1,Dragonball Evolution
2,Loveyatri
3,Race 3
4,Gunday


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [5]:

query = """
        SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int FROM
        Movie
        """

q1 = pd.read_sql_query(query, conn)    
q1.head()


,year_int
0,2018
1,2018
2,2018
3,2012
4,2018


In [6]:
%%time
# Write your sql query below

query = """select distinct(year_int)
        from (SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int FROM
        Movie)as Y
        where ((Y.year_int % 4 = 0) and (Y.year_int % 100 != 0)) or (Y.year_int % 400 = 0)
        
            
"""

q1 = pd.read_sql_query(query, conn)    
q1.head()

Wall time: 5.98 ms


,year_int
0,2012
1,2016
2,2008
3,2004
4,1996


In [7]:

query = """
SELECT * from Genre 
WHERE Name LIKE "%Comedy%"
"""

q1 = pd.read_sql_query(query, conn)    

q1.head()

,index,Name,GID
0,1,"Action, Comedy, Crime",1
1,6,"Comedy, Crime, Thriller",6
2,8,"Comedy, Drama, Family",8
3,10,Comedy,10
4,13,"Action, Comedy, Sci-Fi",13


In [8]:
%%time
query = """
SELECT Person.Name, M.year_int, M.title FROM
(
        (SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int, MID, title FROM
        Movie
        WHERE ((year_int % 4 = 0) AND (year_int % 100 != 0)) OR (year_int % 400 = 0)
       ) as M 
       JOIN M_Director ON M.MID = M_Director.MID
       JOIN M_Genre ON M.MID = M_Genre.MID
       JOIN (SELECT * from Genre WHERE Genre.Name LIKE "%Comedy%") AS g ON M_Genre.GID = g.GID
       JOIN Person ON M_Director.PID = Person.PID
       
 )     
"""

q1 = pd.read_sql_query(query, conn)    
print(q1.shape)
q1.head()

(232, 3)
Wall time: 152 ms


,Name,year_int,title
0,Milap Zaveri,2016,Mastizaade
1,Danny Leiner,2004,Harold & Kumar Go to White Castle
2,Anurag Kashyap,2012,Gangs of Wasseypur
3,Frank Coraci,2004,Around the World in 80 Days
4,Griffin Dunne,2008,The Accidental Husband


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [9]:
%%time
# Write your sql query below

query = """
        SELECT * FROM Movie
        WHERE Movie.title ="Anand"
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(1, 6)
Wall time: 5.99 ms


,index,MID,title,year,rating,num_votes
0,66,tt0066763,Anand,1971,8.8,21616


In [10]:
%%time
# Write your sql query below

query = """
        SELECT * FROM Person
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(37566, 4)
Wall time: 168 ms


,index,PID,Name,Gender
0,0,nm0000288,Christian Bale,Male
1,1,nm0000949,Cate Blanchett,Female
2,2,nm1212722,Benedict Cumberbatch,Male
3,3,nm0365140,Naomie Harris,Female
4,4,nm0785227,Andy Serkis,Male


In [11]:
%%time
# Write your sql query below

query = """
    SELECT PID from
        Movie JOIN M_Cast ON Movie.MID = M_Cast.MID
        WHERE Movie.title = "Anand"
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(17, 1)
Wall time: 240 ms


,PID
0,nm0000821
1,nm0004435
2,nm0080231
3,nm0219946
4,nm0219947


In [12]:
%%time
# Write your sql query below

query = """
        SELECT Person.Name FROM Person
        WHERE Person.PID IN ("nm0219947", "nm0219946")
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(2, 1)
Wall time: 11 ms


,Name
0,Ramesh Deo
1,Seema Deo


In [13]:
%%time
# Write your sql query below

query = """
        SELECT Name FROM Person
        WHERE PID IN
        (SELECT TRIM(PID) from
        Movie JOIN M_Cast ON Movie.MID = M_Cast.MID
        WHERE Movie.title = "Anand"
        )
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(17, 1)
Wall time: 246 ms


,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [14]:
%%time
# Write your sql query below
"""
        (SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int, MID FROM
        Movie WHERE year_int > 1990) AS movies_after_1990
        JOIN M_Cast ON M_Cast.MID = movies_after_1990.MID
"""
query = """
        SELECT Name FROM Person
        WHERE PID IN
        (
        SELECT TRIM(PID) FROM
        (SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int, MID FROM
        Movie WHERE year_int < 1970) AS movies_before_1970
        JOIN M_Cast ON M_Cast.MID = movies_before_1970.MID
        
        INTERSECT
        SELECT TRIM(PID) FROM
        (SELECT CAST(SUBSTR(year, -4) AS INTEGER) AS year_int, MID FROM
        Movie WHERE year_int > 1990) AS movies_after_1990
        JOIN M_Cast ON M_Cast.MID = movies_after_1990.MID
        )
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
q3.head()

(300, 1)
Wall time: 534 ms


,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [15]:
%%time
# Write your sql query below

query = """
        SELECT Person.Name, movie_count FROM
        (SELECT PID, count(Movie.MID) AS movie_count FROM Movie
        JOIN M_Director ON Movie.MID = M_Director.MID
        GROUP BY PID) as M
        JOIN Person ON Person.PID = M.PID
        ORDER BY movie_count DESC
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
q4.head()

(1464, 2)
Wall time: 67.8 ms


,Name,movie_count
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [16]:
%%time
# Write your sql query below

query = """
        SELECT * FROM Person
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(37566, 4)
Wall time: 122 ms


,index,PID,Name,Gender
0,0,nm0000288,Christian Bale,Male
1,1,nm0000949,Cate Blanchett,Female
2,2,nm1212722,Benedict Cumberbatch,Male
3,3,nm0365140,Naomie Harris,Female
4,4,nm0785227,Andy Serkis,Male


In [17]:
q5a.Gender.value_counts()

Male      19924
Female     9416
Name: Gender, dtype: int64

In [18]:
%%time
# Write your sql query below

query = """
        
                SELECT Movie.MID, Movie.Title, Movie.year
                FROM

                (SELECT CASE
                            WHEN Gender = "Male" THEN FALSE
                            WHEN Gender = "Female" THEN TRUE
                       END as gender, PID, Name
                FROM Person) AS P
                JOIN M_Cast ON P.PID = TRIM(M_Cast.PID)
                JOIN Movie ON Movie.MID = M_Cast.MID
                GROUP BY Movie.MID
                HAVING MIN(gender) > 0

        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(4, 3)
Wall time: 455 ms


,MID,title,year
0,tt0272001,Bindhaast,1999
1,tt0354922,Snegithiye,2000
2,tt0375882,Kala Jigar,1939
3,tt8458202,Pihu,I 2018


In [19]:
%%time
# Write your sql query below

query = """
        
        SELECT CAST(SUBSTR(year, -4) AS INTEGER) as year_int, 
         GROUP_CONCAT(Movie.MID) AS all_movies
        FROM Movie
        GROUP BY year_int

        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(78, 2)
Wall time: 7.98 ms


,year_int,all_movies
0,1931,tt0021594
1,1936,"tt0028217,tt0027256,tt0026274"
2,1939,"tt0031580,tt0375882"
3,1941,tt0033616
4,1943,tt0036077


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [20]:
%%time
# Write your sql query below

query = """
        
        
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
q5b.head()

TypeError: 'NoneType' object is not iterable

## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [31]:
%%time
# Write your sql query below

query = """
        SELECT Movie.title, COUNT(DISTINCT(M_Cast.PID)) AS cast_count
        FROM Movie JOIN M_Cast ON Movie.MID = M_Cast.MID
        GROUP By Movie.MID 
        ORDER BY cast_count DESC
        LIMIT 1
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
q6.head()

(1, 2)
Wall time: 345 ms


,title,cast_count
0,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [ ]:
%%time
# Write your sql query below

query = """
        
        
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
q7.head()

## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
# Write your sql query below

query = """
        
        
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head()

## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
# Write your sql query below

query = """
        
        
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()